In [11]:
import pandas as pd
import argparse
import numpy as np
from pathlib import Path
import sys

def generate_sql_inserts(args):
    try:
        # Configurar rutas
        archive_dir = Path('archive')
        output_dir = Path('generated_sql')
        output_dir.mkdir(exist_ok=True)

        # Buscar archivos válidos
        valid_ext = ['.csv', '.xlsx', '.xls']
        input_files = sorted([f for f in archive_dir.iterdir() if f.suffix.lower() in valid_ext])

        if not input_files:
            raise ValueError("No hay archivos en la carpeta 'archive'")

        # Seleccionar primer archivo
        input_file = input_files[0]
        original_name = input_file.stem

        # Generar nombre SQL
        sql_filename = f"{original_name}_generated.sql"
        sql_path = output_dir / sql_filename

        print(f"\n{'='*50}")
        print(f"Archivo origen: {input_file.name}")
        print(f"Archivo SQL destino: {sql_filename}")
        print(f"{'='*50}\n")

        # Leer archivo
        if input_file.suffix.lower() in ['.xlsx', '.xls']:
            df = pd.read_excel(input_file, header=None if args.no_header else 0)
        else:
            df = pd.read_csv(input_file, delimiter=args.delimiter, header=None if args.no_header else 0)

        # Configurar columnas
        if args.no_header:
            df.columns = [f'col{i+1}' for i in range(len(df.columns))]
        else:
            df.columns = [str(col).strip() for col in df.columns]

        # Escapar nombres de columnas para PostgreSQL
        escaped_columns = [f'"{col}"' for col in df.columns]

        # Generar todos los valores
        values_list = []
        for _, row in df.iterrows():
            values = []
            for val, dtype in zip(row, df.dtypes):
                if pd.isna(val):
                    values.append('NULL')
                elif np.issubdtype(dtype, np.integer):
                    values.append(str(int(val)))
                elif np.issubdtype(dtype, np.floating):
                    values.append(f"{float(val):.2f}" if not float(val).is_integer() else str(int(val)))
                elif np.issubdtype(dtype, np.datetime64):
                    values.append(f"'{pd.to_datetime(val).strftime('%Y-%m-%d %H:%M:%S')}'")
                else:
                    safe_val = str(val).replace("'", "''")
                    values.append(f"'{safe_val}'")

            values_list.append(f"({', '.join(values)})")

        # Escribir SQL
        with open(sql_path, 'w', encoding='utf-8') as f:
            f.write(f'-- Generado desde: {input_file.name}\n')
            f.write(f'-- Tabla destino: {args.table_name}\n')
            f.write(f'-- Fecha generación: {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")}\n\n')

            f.write(f"INSERT INTO {args.table_name} ({', '.join(escaped_columns)})\n")
            f.write("VALUES\n")
            f.write(",\n".join(values_list))
            f.write(";\n")

        print(f"✓ SQL generado exitosamente!")
        print(f"✓ Registros procesados: {len(df)}")
        print(f"✓ Ruta del archivo: {sql_path}")

    except Exception as e:
        print(f"\n✗ Error: {str(e)}")
        sys.exit(1)

# Configuración de ejecución
if __name__ == '__main__':
    if 'ipykernel' in sys.modules:
        class Args:
            no_header = False
            delimiter = ','

        args = Args()
        args.table_name = input("\nIngrese el nombre de la tabla SQL: ")
        generate_sql_inserts(args)

    else:
        parser = argparse.ArgumentParser(description='Convertidor a SQL')
        parser.add_argument('--delimiter', default=',', help='Delimitador para CSV')
        parser.add_argument('--no_header', action='store_true', help='Archivo sin encabezados')
        args = parser.parse_args()
        args.table_name = input("\nIngrese el nombre de la tabla SQL: ")
        generate_sql_inserts(args)


Archivo origen: Ventas  del día 21-04-2025.xlsx
Archivo SQL destino: Ventas  del día 21-04-2025_generated.sql

✓ SQL generado exitosamente!
✓ Registros procesados: 49
✓ Ruta del archivo: generated_sql\Ventas  del día 21-04-2025_generated.sql
